<a href="https://colab.research.google.com/github/CodyBurker/W266_Group_Project/blob/master/CNN_Confidence_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on: https://machinelearningmastery.com/develop-word-embedding-model-predicting-movie-review-sentiment/

In [1]:
import pandas as pd
import string
from nltk.corpus import stopwords
import nltk
import numpy as np

In [2]:
!git clone https://github.com/CodyBurker/W266_Group_Project

Cloning into 'W266_Group_Project'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (199/199), done.
remote: Total 224 (delta 97), reused 85 (delta 20), pack-reused 0
Receiving objects: 100% (224/224), 142.11 MiB | 23.96 MiB/s, done.
Resolving deltas: 100% (97/97), done.
Checking out files: 100% (48/48), done.


In [3]:
X_train = pd.read_csv("W266_Group_Project/x_train_stage_1_sampled_yelp_data.csv")
y_train = pd.read_csv("W266_Group_Project/y_train_stage_1_sampled_yelp_data.csv")
X_test = pd.read_csv("W266_Group_Project/x_test_sampled_yelp_data_NEW.csv")
y_test = pd.read_csv("W266_Group_Project/y_test_sampled_yelp_data_NEW.csv")
y_train = np.array(y_train.stars.astype('int')) - 1
y_test = np.array(y_test.stars.astype('int')) - 1

Encode text and pad it.

In [6]:
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['text'])


In [8]:
# Hyper parameters to tune later: maxlen, padding
sentence_len = 1024

In [9]:
from tensorflow import keras
model = keras.models.load_model('W266_Group_Project/CNNModel')
x_train_stage_2 = pd.read_csv("W266_Group_Project/x_train_stage_2_sampled_yelp_data.csv")
x_train_stage_2_tok = tokenizer.texts_to_sequences(x_train_stage_2['text'])
x_train_stage_2_enc = pad_sequences(x_train_stage_2_tok, maxlen=sentence_len, padding='post', truncating = 'post')



In [11]:
stage_2_preds = model.predict(x_train_stage_2_enc)

In [12]:
stage_2_preds[0:10]

array([[5.4274762e-11, 2.7156696e-10, 6.2438602e-07, 1.1437516e-02,
        9.8856187e-01],
       [8.0849964e-04, 1.5695454e-03, 2.1825543e-02, 2.7155662e-01,
        7.0423973e-01],
       [1.5388006e-04, 8.8449102e-04, 2.4960924e-02, 3.8966149e-01,
        5.8433920e-01],
       [2.2776437e-03, 7.5082546e-03, 8.4344968e-02, 5.0288910e-01,
        4.0298003e-01],
       [2.6412055e-04, 2.9147661e-03, 7.7409886e-02, 6.0425001e-01,
        3.1516132e-01],
       [5.8987451e-01, 3.6625132e-01, 4.3304168e-02, 5.3667545e-04,
        3.3230252e-05],
       [2.8363879e-06, 1.9484212e-06, 8.6495951e-05, 2.2184024e-02,
        9.7772467e-01],
       [5.4703225e-05, 1.1042092e-03, 4.7679093e-02, 6.1958158e-01,
        3.3158043e-01],
       [1.3357954e-05, 1.2611100e-04, 8.1348065e-03, 3.7151840e-01,
        6.2020731e-01],
       [3.8825037e-06, 8.4894040e-05, 1.0302837e-02, 4.6546927e-01,
        5.2413911e-01]], dtype=float32)

In [13]:
stage_2_preds_df = pd.DataFrame(stage_2_preds)
stage_2_preds_df.columns = ['CNN1','CNN2','CNN3','CNN4','CNN5']
stage_2_preds_df

,CNN1,CNN2,CNN3,CNN4,CNN5
0,5.427476e-11,2.715670e-10,6.243860e-07,0.011438,0.988562
1,8.084996e-04,1.569545e-03,2.182554e-02,0.271557,0.704240
2,1.538801e-04,8.844910e-04,2.496092e-02,0.389661,0.584339
3,2.277644e-03,7.508255e-03,8.434497e-02,0.502889,0.402980
4,2.641206e-04,2.914766e-03,7.740989e-02,0.604250,0.315161
...,...,...,...,...,...
79995,7.084869e-04,3.085575e-03,5.436052e-02,0.466108,0.475737
79996,9.292719e-08,7.591374e-07,2.654752e-04,0.121828,0.877906
79997,3.046503e-04,3.520773e-04,4.828411e-03,0.132024,0.862490
79998,6.403273e-05,1.884199e-03,8.940139e-02,0.711236,0.197414


In [14]:
# Write processed preds to disk
stage_2_preds_df.to_csv("W266_Group_Project/CNN_Confidence_x_train_stage_2_sampled_yelp_data.csv")

80k Stage 2

In [15]:
x_train_stage_2 = pd.read_csv("W266_Group_Project/x_test_sampled_yelp_data_NEW.csv")
x_train_stage_2_tok = tokenizer.texts_to_sequences(x_train_stage_2['text'])
x_train_stage_2_enc = pad_sequences(x_train_stage_2_tok, maxlen=sentence_len, padding='post', truncating = 'post')

In [16]:
x_train_stage_2_preds = model.predict(x_train_stage_2_enc)

In [17]:
stage_2_preds_df = pd.DataFrame(x_train_stage_2_preds)
stage_2_preds_df.columns = ['CNN1','CNN2','CNN3','CNN4','CNN5']
stage_2_preds_df

,CNN1,CNN2,CNN3,CNN4,CNN5
0,3.192664e-07,8.497344e-07,0.000128,0.056308,0.943563
1,3.967343e-07,6.186232e-06,0.001827,0.273400,0.724766
2,1.501992e-07,1.440981e-06,0.000466,0.157051,0.842481
3,5.104080e-07,1.112177e-06,0.000151,0.056605,0.943243
4,1.618752e-04,3.904097e-04,0.008331,0.196789,0.794327
...,...,...,...,...,...
19995,2.566059e-04,1.955470e-03,0.052774,0.521049,0.423965
19996,6.207200e-01,1.164334e-01,0.054312,0.038678,0.169856
19997,3.409913e-01,2.749851e-01,0.238320,0.090529,0.055175
19998,2.946011e-04,2.487160e-04,0.003161,0.099363,0.896932


In [18]:
stage_2_preds_df.to_csv("W266_Group_Project/CNN_Confidence_x_test_sampled_yelp_data_NEW.csv")

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
